In [ ]:
import os
import fitz  # PyMuPDF
import openai
import pytesseract
import tempfile
import base64
import json
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from tqdm import tqdm
from pdf2image import convert_from_path
from PIL import Image
from openai import OpenAI

# ========== 1. 初始化 OpenAI ==========
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ========== 2. 提取文本：前后20页 ==========
def extract_front_back_text(pdf_path, front_n=10, back_n=10):
    try:
        doc = fitz.open(pdf_path)
        texts = [doc[i].get_text() for i in range(min(front_n, len(doc)))]
        texts += [doc[i].get_text() for i in range(max(0, len(doc) - back_n), len(doc))]
        doc.close()
        full_text = "\n".join(texts)
        if len(full_text.strip()) < 50:
            raise ValueError("Too little text")
        return full_text
    except Exception as e:
        print(f"⚠️ PyMuPDF failed on {pdf_path.name}, switching to OCR...")
        return extract_text_with_ocr(pdf_path)

# ========== 3. OCR 补救 ==========
def extract_text_with_ocr(pdf_path, dpi=300):
    with tempfile.TemporaryDirectory() as path:
        images = convert_from_path(pdf_path, dpi=dpi, output_folder=path)
        text_parts = [pytesseract.image_to_string(img) for img in images[:3] + images[-3:]]
        return "\n".join(text_parts)

# ========== 4. 构造 Prompt ==========
def build_report_year_prompt(text):
    return f"""
You are an expert assistant helping to extract reporting years from corporate reports.

Please complete the following tasks after carefully reading the text below:

---

### Task 1: Identify any **fiscal year**, **reporting period**, or **financial year** expressions.

Common examples include:
- "for the year ended 31 December 2021"
- "reporting period: April 2021 – March 2022"
- "2011–14" or "FY2017"
- "2020 ESG highlights"
- "2013 Corporate Report"

---

### Task 2: Normalize each valid expression into a standardized format:

1. If the expression is a **single year**, like `"2013"` or `"FY2020"`, output `"2013"` or `"2020"` as-is.
2. If it is a **range of years only**, like `"2014–15"`, output `"2014 to 2015"`. Do **not** add months or days.
3. If full dates are mentioned (e.g., `"1st April 2020 – 31st March 2021"`), remove ordinal suffixes (`1st`, `2nd`, etc.) and format as:
   `"1 April 2020 to 31 March 2021"`
4. Always convert dashes ("–" or "-") to "to".
5. If month is provided but not day (e.g., `"April 2020 – March 2021"`), convert to `"April 2020 to March 2021"`.

---

### Task 3: Return a JSON in the following format:

If a valid reporting year expression is found:

{{
  "normalized_report_year": "your normalized result here",
  "original_expression": "copy the original matched expression",
  "source": "e.g. Page 1, main text"
}}

If no valid expression is found:

{{
  "normalized_report_year": null,
  "original_expression": null,
  "source": "NOT FOUND"
}}

Only output the JSON object. Do not add explanations or commentary.

---

Text:
{text}
"""

# ========== 5. Vision 模型辅助 ==========
def encode_image_to_base64(pil_image):
    with tempfile.NamedTemporaryFile(suffix=".png") as f:
        pil_image.save(f.name, format="PNG")
        with open(f.name, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode("utf-8")

def extract_year_from_vision(pdf_path, client, page_limit=3):
    try:
        images = convert_from_path(pdf_path, dpi=200)
        for i, img in enumerate(images[:page_limit]):
            b64 = encode_image_to_base64(img)
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "user", "content": [
                        {"type": "text", "text": f"Please extract the fiscal year or reporting period from page {i+1}. Please return in this JSON format:\n"
                                                 "{\n  \"normalized_report_year\": \"...\",\n  \"original_expression\": \"...\",\n  \"source\": \"Page {i+1}, image-based\"\n}"},
                        {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{b64}"}}
                    ]}
                ],
                max_tokens=300
            )
            content = response.choices[0].message.content
            parsed = json.loads(content.strip()) if content.strip().startswith("{") else eval(content.strip())
            if parsed.get("normalized_report_year"):
                return parsed
    except Exception as e:
        return {
            "normalized_report_year": None,
            "original_expression": None,
            "source": f"Vision ERROR: {e}"
        }
    return {
        "normalized_report_year": None,
        "original_expression": None,
        "source": "Vision NOT FOUND"
    }

# ========== 6. 主函数：先文本，再 Vision ==========
def extract_report_year(pdf_path, client):
    try:
        text = extract_front_back_text(pdf_path)
        prompt = build_report_year_prompt(text)
        response = client.chat.completions.create(
            model="gpt-4.1-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
            max_tokens=500
        )
        content = response.choices[0].message.content
        parsed = json.loads(content.strip()) if content.strip().startswith("{") else eval(content.strip())
        if parsed.get("normalized_report_year"):
            return parsed
    except Exception as e:
        print(f"⚠️ GPT-4.1-mini failed on {Path(pdf_path).name}, fallback to Vision...")
    return extract_year_from_vision(pdf_path, client)


from joblib import Parallel, delayed

# 多线程处理函数
def process_pdf(pdf_path):
    try:
        out = extract_report_year(str(pdf_path), client)
        out["filename"] = pdf_path.name
    except Exception as e:
        out = {
            "filename": pdf_path.name,
            "normalized_report_year": None,
            "original_expression": None,
            "source": f"ERROR: {e}"
        }
    return out

def batch_extract_years_multithread(pdf_dir, output_csv="results/extracted_report_years_formal1.csv", n_jobs=2):
    pdf_dir = Path(pdf_dir)
    pdf_files = sorted(pdf_dir.glob("*.pdf"))
    os.makedirs(Path(output_csv).parent, exist_ok=True)

    # ✅ 并行处理
    results = Parallel(n_jobs=n_jobs, prefer="threads")(
        delayed(process_pdf)(pdf) for pdf in tqdm(pdf_files, desc="📄 Processing PDFs")
    )

    df = pd.DataFrame(results)
    df.to_csv(output_csv, index=False)
    print(f"\n✅ Extraction complete! Results saved to: {output_csv}")

# ========== 8. 运行 ==========
if __name__ == "__main__":
    batch_extract_years_multithread("pdf_folder")  

📄 Processing PDFs:   6%|▋         | 82/1277 [01:21<19:22,  1.03it/s]

⚠️ GPT-4.1-mini failed on Armstrong_Flooring_Inc_SustainabilityReport-2020_kot54emv.pdf, fallback to Vision...


📄 Processing PDFs:   7%|▋         | 84/1277 [01:21<13:01,  1.53it/s]

⚠️ GPT-4.1-mini failed on Arvind_Ltd_Arvind_AR_2022-23_0_iwp4673c.pdf, fallback to Vision...


📄 Processing PDFs:   9%|▉         | 115/1277 [01:50<12:15,  1.58it/s]

...

⚠️ GPT-4.1-mini failed on Unknown_23076_Whitbread_AR2020_web_0v2mxh4f.pdf, fallback to Vision...


📄 Processing PDFs: 100%|█████████▉| 1274/1277 [26:17<00:04,  1.50s/it]

⚠️ GPT-4.1-mini failed on Unknown_adbi-managing-transition-low-carbon-economy_087is5zy.pdf, fallback to Vision...
⚠️ GPT-4.1-mini failed on Unknown_adp07-sus-fr_95qx6prh.pdf, fallback to Vision...


📄 Processing PDFs: 100%|██████████| 1277/1277 [26:17<00:00,  1.24s/it]



✅ Extraction complete! Results saved to: results/extracted_report_years_formal1.csv


In [3]:
import pandas as pd

# 读取文件
anno_df = pd.read_excel("check/rfyear_annotation.xlsx")
result_df = pd.read_csv("results/extracted_report_years_formal1.csv")

# 清洗文件名列
anno_df["pdf_name_clean"] = anno_df["pdf_name"].str.strip().str.lower()
result_df["filename_clean"] = result_df["filename"].str.strip().str.lower()

# 合并两个表
merged_df = pd.merge(
    anno_df,
    result_df,
    left_on="pdf_name_clean",
    right_on="filename_clean",
    how="left"
)

# 年份对比函数
def match_year(human, gpt):
    if pd.isna(human) or pd.isna(gpt):
        return "missing"
    return "match" if str(human).strip().lower() == str(gpt).strip().lower() else "mismatch"

# 添加匹配结果
merged_df["match_result"] = merged_df.apply(
    lambda row: match_year(row["chosen_rfyear"], row["normalized_report_year"]),
    axis=1
)

# 仅保留有用字段
final_df = merged_df[[
    "pdf_name", "chosen_rfyear", "normalized_report_year", "match_result"
]]

# 保存结果
# final_df.to_csv("rfyear_comparison_results_minimal.csv", index=False)

# 输出统计
print("✅ 已保存简洁对比结果为 rfyear_comparison_results_minimal.csv")
print("📊 匹配情况统计：")
print(final_df["match_result"].value_counts())

✅ 已保存简洁对比结果为 rfyear_comparison_results_minimal.csv
📊 匹配情况统计：
match_result
missing     389
mismatch    115
match        97
Name: count, dtype: int64


In [10]:
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

# 1. 读取两个文件
gpt_df = pd.read_csv("results/extracted_report_years_formal1.csv")  # 提取结果
label_df = pd.read_excel("check/rfyear_annotation.xlsx")         # 人工标注文件

# 2. 重命名列以便对齐合并
gpt_df.rename(columns={"filename": "pdf_name", "report_year": "normalized_report_year"}, inplace=True)

# 3. 合并两个表（inner join，只保留两个都有的 pdf）
merged = pd.merge(label_df, gpt_df, on="pdf_name", how="inner")

# 4. 标准化字符串格式（去除空格大小写等）
merged["normalized_report_year"] = merged["normalized_report_year"].astype(str).str.strip().str.lower()
merged["chosen_rfyear"] = merged["chosen_rfyear"].astype(str).str.strip().str.lower()

# 5. 定义匹配逻辑（完全匹配即可）
merged["match"] = merged["normalized_report_year"] == merged["chosen_rfyear"]

# 6. 统计指标
total = len(merged)
correct = merged["match"].sum()
accuracy = correct / total

print(f"✅ 合并样本数：{total}")
print(f"✅ 匹配正确数：{correct}")
print(f"✅ 准确率（accuracy）：{accuracy:.2%}")

# 7. 可选：保存对比结果
merged[["pdf_name", "chosen_rfyear", "normalized_report_year", "match"]].to_csv("eval/year_comparison1.csv", index=False)

✅ 合并样本数：224
✅ 匹配正确数：97
✅ 准确率（accuracy）：43.30%


In [15]:
import pandas as pd
import re

# === 1. 加载数据 ===
gpt_df = pd.read_csv("results/extracted_report_years_formal1.csv")  # GPT提取结果
label_df = pd.read_excel("check/rfyear_annotation.xlsx")            # 标注结果

# === 2. 重命名列对齐、合并（按文件名）===
gpt_df.rename(columns={"filename": "pdf_name", "report_year": "normalized_report_year"}, inplace=True)
merged = pd.merge(label_df, gpt_df, on="pdf_name", how="inner")

# === 3. 清洗文本：空值转空串，大小写、空格、换行处理 ===
def clean_text(s):
    if pd.isna(s): return ""
    return str(s).strip().lower().replace("\n", " ")

merged["normalized_report_year"] = merged["normalized_report_year"].apply(clean_text)
merged["chosen_rfyear"] = merged["chosen_rfyear"].apply(clean_text)

# === 4. 定义辅助函数 ===
import re

def normalize_year_text(s):
    if not s:
        return ""
    s = s.lower()

    # 去掉标点与多余字符
    s = re.sub(r'[\(\)\[\],;:]', ' ', s)
    s = re.sub(r'\s+', ' ', s)
    s = re.sub(r'[^a-z0-9\s\-/–]', ' ', s)

    # 替换 FY 缩写和连接符为 to
    s = s.replace("–", "to").replace("-", "to").replace("/", "to")
    s = re.sub(r'\bfy\b', 'fiscal year', s)
    s = re.sub(r'\bfy(\d{2})\b', lambda m: f"20{m.group(1)}", s)  # fy20 → 2020
    s = re.sub(r'\bfy\s*(\d{4})\b', r'\1', s)

    # 展开缩写年份如 2022–23 → 2022 to 2023
    s = re.sub(r'\b(20\d{2})\s*to\s*(\d{2})\b', lambda m: f"{m.group(1)} to 20{m.group(2)}", s)
    s = re.sub(r'\b(20\d{2})\s*[\-–]\s*(\d{2})\b', lambda m: f"{m.group(1)} to 20{m.group(2)}", s)
    s = re.sub(r'\b(20\d{2})[\-–](20\d{2})\b', r'\1 to \2', s)
    s = re.sub(r'\b(20\d{2})\s+to\s+(20\d{2})\b', r'\1 to \2', s)

    # 清理序数词，如 1st, 2nd, 3rd
    s = re.sub(r'\b(\d{1,2})(st|nd|rd|th)\b', r'\1', s)

    # 统一日期格式，如 "31 march 2023" → "31 march 2023"
    s = s.strip()
    return s

def extract_years(s):
    return sorted(set(re.findall(r'\b(20\d{2}|19\d{2})\b', s)))

def is_fuzzy_match(a, b):
    a_norm = normalize_year_text(a)
    b_norm = normalize_year_text(b)
    if a_norm == b_norm:
        return True
    a_years = extract_years(a_norm)
    b_years = extract_years(b_norm)
    if not a_years or not b_years:
        return False
    # 年份集合完全一致 or 包含关系
    if set(a_years) == set(b_years):
        return True
    if len(a_years) == 1 and a_years[0] in b_years:
        return True
    if len(b_years) == 1 and b_years[0] in a_years:
        return True
    # 仅一个年份时近似判断
    if len(a_years) == 1 and len(b_years) == 1 and abs(int(a_years[0]) - int(b_years[0])) <= 1:
        return True
    return False

# === 5. 应用模糊匹配函数 ===
merged["fuzzy_match"] = merged.apply(lambda row: is_fuzzy_match(row["normalized_report_year"], row["chosen_rfyear"]), axis=1)

# === 6. 计算准确率 ===
total = len(merged)
correct = merged["fuzzy_match"].sum()
accuracy = correct / total

print(f"✅ 合并样本数：{total}")
print(f"✅ 匹配正确数：{correct}")
print(f"✅ 模糊匹配准确率（Fuzzy Accuracy）：{accuracy:.2%}")

# === 7. 保存结果 ===
merged[["pdf_name", "chosen_rfyear", "normalized_report_year", "fuzzy_match"]].to_csv("eval/year_comparison3.csv", index=False)

✅ 合并样本数：224
✅ 匹配正确数：178
✅ 模糊匹配准确率（Fuzzy Accuracy）：79.46%


In [17]:
import pandas as pd
import re

# === 1. 加载数据 ===
gpt_df = pd.read_csv("results/extracted_report_years_formal1.csv")  # GPT提取结果
label_df = pd.read_excel("check/rfyear_annotation.xlsx")            # 标注结果

# === 2. 重命名列对齐、合并（按文件名）===
gpt_df.rename(columns={"filename": "pdf_name", "report_year": "normalized_report_year"}, inplace=True)
merged = pd.merge(label_df, gpt_df, on="pdf_name", how="inner")

# === 3. 清洗文本：空值转空串，大小写、空格、换行处理 ===
def clean_text(s):
    if pd.isna(s): return ""
    return str(s).strip().lower().replace("\n", " ")

merged["normalized_report_year"] = merged["normalized_report_year"].apply(clean_text)
merged["chosen_rfyear"] = merged["chosen_rfyear"].apply(clean_text)

# === 4. 定义辅助函数 ===
import re

def normalize_year_text(s):
    if not s:
        return ""
    s = s.lower().strip()
    
    # 替换常见连字符为统一格式
    s = s.replace("–", " to ").replace("-", " to ").replace("/", " to ")

    # 清除无效字符
    s = re.sub(r'[\(\)\[\],;:]', ' ', s)
    s = re.sub(r'\s+', ' ', s)

    # fy 缩写处理：fy2020 或 fy 2020 → 2020
    s = re.sub(r'\bfy\s*(\d{4})\b', r'\1', s)                   # fy 2020 → 2020
    s = re.sub(r'\bfy\s*(\d{2})\b', lambda m: f"20{m.group(1)}", s)  # fy 19 → 2019

    # 区间格式处理：2020 to 21 → 2020 to 2021
    s = re.sub(r'\b(20\d{2})\s+to\s+(\d{2})\b', lambda m: f"{m.group(1)} to 20{m.group(2)}", s)

    # 区间格式处理：2020–2021、2020-2021、fy2020 to 2021 → 2020 to 2021
    s = re.sub(r'\b(20\d{2})\s*to\s*(20\d{2})\b', r'\1 to \2', s)
    s = re.sub(r'\b(20\d{2})\s+to\s+20\d{2}', r'\g<0>', s)

    # 清理序数词
    s = re.sub(r'\b(\d{1,2})(st|nd|rd|th)\b', r'\1', s)

    return s.strip()

def extract_years(s):
    return sorted(set(re.findall(r'\b(20\d{2}|19\d{2})\b', s)))

def is_fuzzy_match(a, b):
    a_norm = normalize_year_text(a)
    b_norm = normalize_year_text(b)
    if a_norm == b_norm:
        return True

    # 提取年份集合
    a_years = extract_years(a_norm)
    b_years = extract_years(b_norm)

    if not a_years or not b_years:
        return False

    # 排序后再比较
    a_sorted = sorted(set(a_years))
    b_sorted = sorted(set(b_years))

    # 完全一致或包含
    if a_sorted == b_sorted:
        return True
    if len(a_sorted) == 1 and a_sorted[0] in b_sorted:
        return True
    if len(b_sorted) == 1 and b_sorted[0] in a_sorted:
        return True

    # 仅一个年份时允许±1
    if len(a_sorted) == 1 and len(b_sorted) == 1 and abs(int(a_sorted[0]) - int(b_sorted[0])) <= 1:
        return True

    return False

# === 5. 应用模糊匹配函数 ===
merged["fuzzy_match"] = merged.apply(lambda row: is_fuzzy_match(row["normalized_report_year"], row["chosen_rfyear"]), axis=1)

# === 6. 计算准确率 ===
total = len(merged)
correct = merged["fuzzy_match"].sum()
accuracy = correct / total

print(f"✅ number of combine samples：{total}")
print(f"✅ correct matches：{correct}")
print(f"✅ Fuzzy Accuracy：{accuracy:.2%}")

# === 7. 保存结果 ===
merged[["pdf_name", "chosen_rfyear", "normalized_report_year", "fuzzy_match"]].to_csv("eval/year_comparison3.csv", index=False)

✅ number of combine samples：224
✅ correct matches：189
✅ Fuzzy Accuracy：84.38%
